In [11]:
!pip install evaluate seqeval transformers datasets

In [12]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification, TrainingArguments, AutoModelForTokenClassification
from torch.utils.data import DataLoader
import torch
import numpy as np
from sklearn.metrics import classification_report
from evaluate import load

# Print a message to confirm successful imports
print("Libraries imported successfully.")

Libraries imported successfully.


In [13]:
train_file = "/content/train.tsv"
test_file = "/content/test.tsv"
print(f"Train file: {train_file}")
print(f"Test file: {test_file}")

Train file: /content/train.tsv
Test file: /content/test.tsv


In [14]:
def read_tsv(file_path):
    sentences, labels = [], []
    sentence, label = [], []

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:  # Empty line separates sentences
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                sentence, label = [], []
            else:
                token, tag = line.split("\t")
                sentence.append(token)
                label.append(tag)

        # Add the last sentence if the file does not end with a blank line
        if sentence:
            sentences.append(sentence)
            labels.append(label)

    return sentences, labels

# Load datasets
train_sentences, train_labels = read_tsv(train_file)
test_sentences, test_labels = read_tsv(test_file)

# Print a sample for verification
print("Sample Sentence:", train_sentences[0])
print("Sample Labels:", train_labels[0])

Sample Sentence: ['Identification', 'of', 'APC2', ',', 'a', 'homologue', 'of', 'the', 'adenomatous', 'polyposis', 'coli', 'tumour', 'suppressor', '.']
Sample Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Disease', 'I-Disease', 'I-Disease', 'E-Disease', 'O', 'O']


In [15]:
unique_labels = set(tag for label_seq in train_labels + test_labels for tag in label_seq)
label_to_id = {label: idx for idx, label in enumerate(sorted(unique_labels))}
id_to_label = {idx: label for label, idx in label_to_id.items()}

print("Label to ID mapping:", label_to_id)

Label to ID mapping: {'B-Disease': 0, 'E-Disease': 1, 'I-Disease': 2, 'O': 3, 'S-Disease': 4}


In [16]:
from datasets import Dataset

def preprocess_data(sentences, labels, label_to_id):
    data = {"tokens": [], "labels": []}

    for sentence, label_seq in zip(sentences, labels):
        # Convert IOBES labels to numeric IDs
        label_ids = [label_to_id[label] for label in label_seq]
        data["tokens"].append(sentence)
        data["labels"].append(label_ids)

    return Dataset.from_dict(data)

# Preprocess train and test datasets
train_data = preprocess_data(train_sentences, train_labels, label_to_id)
test_data = preprocess_data(test_sentences, test_labels, label_to_id)

# Print dataset info
print(train_data)

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 5424
})


In [17]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True,
    )

    all_labels = []
    for i, labels in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None

        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)
            elif word_idx != previous_word_idx:
                aligned_labels.append(labels[word_idx] if word_idx < len(labels) else -100)
            else:
                aligned_labels.append(-100)
            previous_word_idx = word_idx

        all_labels.append(aligned_labels)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

# Initialize tokenizer
model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize datasets
tokenized_train_data = train_data.map(tokenize_and_align_labels, batched=True)
tokenized_test_data = test_data.map(tokenize_and_align_labels, batched=True)

print(tokenized_train_data[0])

Map:   0%|          | 0/5424 [00:00<?, ? examples/s]

Map:   0%|          | 0/940 [00:00<?, ? examples/s]

{'tokens': ['Identification', 'of', 'APC2', ',', 'a', 'homologue', 'of', 'the', 'adenomatous', 'polyposis', 'coli', 'tumour', 'suppressor', '.'], 'labels': [-100, 3, 3, 3, -100, -100, -100, 3, 3, 3, -100, -100, 3, 3, 0, -100, -100, -100, 2, -100, -100, -100, 2, -100, 1, -100, 3, -100, 3, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], 'input_ids': [101, 9117, 1104, 170, 1643, 1665, 1477, 117, 170, 16358, 3702, 12733, 1104, 1103, 8050, 26601, 21943, 

In [18]:
metric = load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (-100) in predictions and labels
    true_predictions = [
        [id_to_label[pred] for pred, label in zip(prediction, label_seq) if label != -100]
        for prediction, label_seq in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[label] for pred, label in zip(prediction, label_seq) if label != -100]
        for prediction, label_seq in zip(predictions, labels)
    ]

    # Compute metrics
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Load model
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(label_to_id)
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Import the Trainer class
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Handles evaluation at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,  # Keeps only the last 2 checkpoints
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-19-8ef8dd2c3ef2>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.093800,0.051476,0.807400,0.886458,0.845084,0.982354
2,0.031600,0.059673,0.848516,0.863542,0.855963,0.985050
3,0.009800,0.067542,0.854675,0.876042,0.865226,0.985540
4,0.004400,0.085890,0.826130,0.876042,0.850354,0.984110
5,0.002000,0.091093,0.842158,0.878125,0.859765,0.985090


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

TrainOutput(global_step=3390, training_loss=0.023446776838780856, metrics={'train_runtime': 827.8285, 'train_samples_per_second': 32.76, 'train_steps_per_second': 4.095, 'total_flos': 1771640061235200.0, 'train_loss': 0.023446776838780856, 'epoch': 5.0})

In [20]:
from sklearn.metrics import classification_report

# Evaluate model
predictions, labels, _ = trainer.predict(tokenized_test_data)
predictions = np.argmax(predictions, axis=2)

# Align predictions and labels
true_predictions = [
    [id_to_label[pred] for pred, label in zip(prediction, label_seq) if label != -100]
    for prediction, label_seq in zip(predictions, labels)
]
true_labels = [
    [id_to_label[label] for pred, label in zip(prediction, label_seq) if label != -100]
    for prediction, label_seq in zip(predictions, labels)
]

# Generate classification report
report = classification_report(
    [label for sublist in true_labels for label in sublist],
    [label for sublist in true_predictions for label in sublist],
    zero_division=0
)
print(report)

              precision    recall  f1-score   support

   B-Disease       0.86      0.88      0.87       537
   E-Disease       0.92      0.94      0.93       537
   I-Disease       0.85      0.91      0.88       550
           O       1.00      0.99      0.99     22434
   S-Disease       0.87      0.90      0.88       423

    accuracy                           0.99     24481
   macro avg       0.90      0.92      0.91     24481
weighted avg       0.99      0.99      0.99     24481

